# CARTOLA FC - 2020

Cartola FC é um jogo eletrônico de futebol no estilo fantasy game, um jogo fictício no qual as pessoas montam seus times com jogadores de futebol da vida real, lançado no ano de 2004. Foi criado e é mantido pelo site Globo.com e promovido pelo canal de TV por assinatura SporTV e também pela Globo.

<hr>

## Definição do Problema

### Previsão de Escalação

Realizar previsão da pontuação dos jogadores do Cartola FC.

<hr>

## Conjunto de dados

In [43]:
# analise de dados
import pandas as pd
import numpy as np
import random as rnd

# Requests e Elementos HTML
import re                          # Expressão regulares
import requests                    # Acessar páginas da internet
from bs4 import BeautifulSoup      # Raspar elementos de páginas da internet

# visualização
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

# metricas 
from sklearn.metrics import classification_report

### Captura dos dados 

Os dados são baixados diretamente do diretório do do GitHub: https://github.com/henriquepgomide/caRtola/tree/master/data/2020

In [44]:

# URL com caminho do repositório
URL = 'https://github.com/henriquepgomide/caRtola/tree/master/data/2020'
html = requests.get(URL)
# Criar objeto BeautifulSoup para raspar urls 
soup = BeautifulSoup(html.text, 'lxml')

In [45]:
# Formatar HTML para extrair os dados
result = []
for tag in soup.find_all('a', href=True):
    result.append(tag)
result[70:90:2]

[<a class="f6 link-gray text-mono ml-2 d-none d-lg-inline" data-pjax="" href="/henriquepgomide/caRtola/commit/fe70dcf565ce007105f4eccd386be7d0b7c43f96">fe70dcf</a>,
 <a class="link-gray-dark text-bold" data-pjax="true" href="/henriquepgomide/caRtola/commit/fe70dcf565ce007105f4eccd386be7d0b7c43f96">fix round 10 file</a>,
 <a class="d-none js-permalink-shortcut" data-hotkey="y" href="/henriquepgomide/caRtola/tree/fe70dcf565ce007105f4eccd386be7d0b7c43f96/data/2020">Permalink</a>,
 <a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/tree/master/data/2020/team-features" id="015cc237790ac744a369fe0a639f3b4b-40cbb9d10b172966a10d0cc0beb858725ca89218" title="team-features">team-features</a>,
 <a class="js-navigation-open link-gray-dark" href="/henriquepgomide/caRtola/tree/master/data/2020/team-rankings" id="4c3fa43e2bc40689660a8b10cb303c0a-e9fa230a75ec759e35a03d712be5e14549625e2f" title="team-rankings">team-rankings</a>,
 <a class="js-navigation-open link-gray-dark" href

In [239]:
regex =  '\.csv$'

dict_of_files = {}                                                          # Criar dicionário vazio
for tag in soup.find_all('a', attrs={'href': re.compile(regex)}):           # Encontrar tags de nosso interesse
    href_str = tag.get('href')                                              
    file_name = re.sub('/henriquepgomide/caRtola/blob/master/data/2020/',   # Substituir padrão por nada
                       '', 
                       href_str)
    
    file_url = re.sub('/henriquepgomide/caRtola/blob/master/data/2020/',    # Substituir padrão por links para arquivos raw
                    'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/', 
                    href_str)

    dict_of_files[file_name] = file_url

In [240]:
dict(list(dict_of_files.items())) # Ignore este código horrível

{'2020-medias-jogadores.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/2020-medias-jogadores.csv',
 '2020_partidas.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/2020_partidas.csv',
 'rodada-1.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-1.csv',
 'rodada-10.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-10.csv',
 'rodada-11.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-11.csv',
 'rodada-2.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-2.csv',
 'rodada-3.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-3.csv',
 'rodada-4.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rodada-4.csv',
 'rodada-5.csv': 'https://raw.githubusercontent.com/henriquepgomide/caRtola/master/data/2020/rod

In [282]:
# Ler os dataframes das rodadas
list_of_dataframes = []
for key, item in list(dict_of_files.items())[2:14]:
    df = pd.read_csv(item)
    df['rodada'] = key
    list_of_dataframes.append(df)

In [283]:
# Combinar lista de DataFrames criados
df_jogadores = pd.concat(list_of_dataframes)
df_jogadores.shape


(7854, 35)

In [284]:
# Ler os dataframes das partidas 
for key, item in list(dict_of_files.items())[1:2]:
    df_jogos = pd.read_csv(item)


In [285]:
df_jogos.shape

(119, 6)

In [286]:
df_jogadores.tail(2)

,Unnamed: 0,atletas.nome,atletas.slug,atletas.apelido,atletas.foto,atletas.atleta_id,atletas.rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,atletas.clube.id.full.name,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,rodada,DP
710,711,Caio Alan Tem Catem Gonçalves,caio,Caio,https://s.glbimg.com/es/sde/f/2018/03/01/87be2...,99460,9,293,gol,Nulo,0.0,1.0,0.00,0.00,0,Athlético-PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-9.csv,NaN
711,712,Luiz Henrique André Rosa da Silva,luiz-henrique,Luiz Henrique,https://s.glbimg.com/es/sde/f/2020/08/11/4a557...,109282,9,266,ata,Nulo,0.2,1.5,-0.48,1.01,7,Fluminense,5.0,1.0,NaN,14.0,NaN,4.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rodada-9.csv,NaN


In [287]:
df_jogadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7854 entries, 0 to 711
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  7854 non-null   int64  
 1   atletas.nome                7854 non-null   object 
 2   atletas.slug                7854 non-null   object 
 3   atletas.apelido             7854 non-null   object 
 4   atletas.foto                7792 non-null   object 
 5   atletas.atleta_id           7854 non-null   int64  
 6   atletas.rodada_id           7854 non-null   int64  
 7   atletas.clube_id            7854 non-null   int64  
 8   atletas.posicao_id          7854 non-null   object 
 9   atletas.status_id           7854 non-null   object 
 10  atletas.pontos_num          7854 non-null   float64
 11  atletas.preco_num           7854 non-null   float64
 12  atletas.variacao_num        7854 non-null   float64
 13  atletas.media_num           7854 n

In [288]:
df_jogos.tail(2)

,date,home_team,away_team,home_score,away_score,round
117,2020-09-27,354,290,NaN,NaN,12
118,2020-09-27,373,263,NaN,NaN,12


In [289]:
df_jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        119 non-null    object 
 1   home_team   119 non-null    int64  
 2   away_team   119 non-null    int64  
 3   home_score  103 non-null    float64
 4   away_score  103 non-null    float64
 5   round       119 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 5.7+ KB


### Primeiro ajuste do dataset

In [290]:
# Remover colunas atletas.foto e rodada
df_jogadores.drop(['atletas.foto', 'rodada', 'atletas.slug', 'atletas.nome', 'atletas.clube.id.full.name'], axis=1, inplace=True)

In [291]:
# Remover linhas cuja coluna rodada_id esteja NaN
df_jogadores = df_jogadores[df_jogadores['atletas.rodada_id'].notnull()]

# Remover coluna Unnamed
df_jogadores = df_jogadores.loc[:, ~df_jogadores.columns.str.contains('^Unnamed')]

In [292]:
# Remover técnicos do dataset
df_jogadores = df_jogadores[df_jogadores['atletas.posicao_id'] != 'tec']

In [311]:
# Remover jogadores contundios e suspensos da rodada
check_colunas_null = ['A','CA','CV','DD','DP','DS','FC','FD','FF','FS','FT','G','GC','GS','I','PI','PP','SG']
df_jogadores = df_jogadores.dropna(subset=check_colunas_null, how='all')

In [322]:
 # preencher com 0 colunas NaN
 df_jogadores.fillna(0, inplace=True)


In [323]:
df_jogadores.shape

(4448, 33)

In [257]:
# Verificar dados NaN
df_jogadores.isna().sum()

atletas.apelido            0
atletas.atleta_id          0
atletas.rodada_id          0
atletas.clube_id           0
atletas.posicao_id         0
atletas.status_id          0
atletas.pontos_num         0
atletas.preco_num          0
atletas.variacao_num       0
atletas.media_num          0
atletas.jogos_num          0
FF                      2215
FS                      1168
G                       3794
PI                       232
CA                      2818
FC                       953
DS                       964
FT                      4339
DD                      4442
GS                      4430
FD                      2788
GC                      4668
SG                      3600
A                       3963
I                       3686
CV                      4540
PP                      4659
DP                      4686
dtype: int64

In [258]:
df_jogos.shape

(119, 6)

In [259]:
# Verificar dados NaN no dataset das Partidas
df_jogos.isna().sum()

date           0
home_team      0
away_team      0
home_score    16
away_score    16
round          0
dtype: int64

In [297]:
# Remover linhas NaN pelo home score
df_jogos.dropna(subset=['home_score'], inplace=True)

In [298]:
# Renomear colunas que possuem a número da rodada para rodada_id em ambos dataset
df_jogos.rename(columns={'round':'rodada_id'}, inplace=True)
df_jogadores.rename(columns={'atletas.rodada_id':'rodada_id'}, inplace=True)

In [299]:
df_jogadores.tail(3)

,atletas.apelido,atletas.atleta_id,rodada_id,atletas.clube_id,atletas.posicao_id,atletas.status_id,atletas.pontos_num,atletas.preco_num,atletas.variacao_num,atletas.media_num,atletas.jogos_num,FF,FS,G,PI,CA,FC,DS,FT,DD,GS,FD,GC,SG,A,I,CV,PP,DP
709,Cayo Tenório,107367,9,267,lat,Nulo,0.0,4.88,0.00,4.33,4,1.0,7.0,NaN,17.0,2.0,5.0,5.0,NaN,NaN,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN
710,Caio,99460,9,293,gol,Nulo,0.0,1.00,0.00,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711,Luiz Henrique,109282,9,266,ata,Nulo,0.2,1.50,-0.48,1.01,7,5.0,1.0,NaN,14.0,NaN,4.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
# Procura qual adversário por rodade e atleta
def getAdversario(clube_id, rodada_id):
    advFora = df_jogos.loc[(df_jogos['home_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['away_team']
    advCasa = df_jogos.loc[(df_jogos['away_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['home_team']
    if (advFora is not None) and (advFora.values > 0):
        return int(advFora.values[0])
    else :
         if (advCasa is not None) and (advCasa.values > 0):
               return int(advCasa.values[0])


In [301]:
# procura se o jogo foi em casa ou f
def getJogoForaEmCasa(clube_id, rodada_id):
    jogouEmCasa = df_jogos.loc[(df_jogos['home_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['home_team']
    jogouFora = df_jogos.loc[(df_jogos['away_team'] == clube_id) & (df_jogos['rodada_id'] == rodada_id)]['away_team']
    if (jogouEmCasa is not None) and (jogouEmCasa.values > 0):
        return 0
    else :
         if (jogouFora is not None) and (jogouFora.values > 0):
               return 1


In [302]:
def gerarColunasGols(linha_df):
    jogoRealizadoEmCasa = df_jogos.loc[(df_jogos['home_team'] == linha_df['atletas.clube_id']) & (df_jogos['rodada_id'] == linha_df['rodada_id'])]
    jogoRealizadoFora = df_jogos.loc[(df_jogos['away_team'] == linha_df['atletas.clube_id']) & (df_jogos['rodada_id'] == linha_df['rodada_id'])]
    if not jogoRealizadoEmCasa.empty:
        linha_df['golFavor'] = jogoRealizadoEmCasa['home_score'].values[0]
        linha_df['golContra'] = jogoRealizadoEmCasa['away_score'].values[0]
    else:
        if not jogoRealizadoFora.empty:
            linha_df['golFavor'] = jogoRealizadoFora['away_score'].values[0]
            linha_df['golContra'] = jogoRealizadoFora['home_score'].values[0]
    return linha_df
        

In [303]:
# Gera novas colunas com os gols do time a favor ou contra para cada rodada
df_jogadores = df_jogadores.apply(lambda x: gerarColunasGols(x), axis=1)

In [304]:
# Adversário do time/jogador
df_jogadores['adversario'] = df_jogadores.apply(lambda x: getAdversario(x['atletas.clube_id'], x['rodada_id']), axis=1)

In [305]:
# Jogo fora ou em casa
df_jogadores['jogoForaOuCasa'] = df_jogadores.apply(lambda x: getJogoForaEmCasa(x['atletas.clube_id'], x['rodada_id']), axis=1)

In [306]:
pd.set_option('display.max_columns', None)
df_jogadores.tail(30)

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
677,2.0,1.0,NaN,NaN,NaN,4.0,7.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,26.0,NaN,1.0,Isla,69345,262,4,2.60,-0.4,lat,8.17,Dúvida,-0.27,1.0,2.0,9,266.0,1.0
679,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,Fabinho,101729,293,3,-0.47,1.0,ata,0.77,Provável,0.00,1.0,1.0,9,263.0,0.0
680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rodrigo Freitas,103261,276,0,0.00,0.0,zag,1.00,Nulo,0.00,1.0,1.0,9,280.0,0.0
681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Haydar,108385,280,0,0.00,0.0,zag,2.00,Nulo,0.00,1.0,1.0,9,276.0,1.0
682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gabriel Bernard,109618,373,0,0.00,0.0,gol,1.00,Nulo,0.00,1.0,2.0,9,267.0,1.0
683,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,Robinho,62104,284,2,2.85,0.0,mei,5.01,Provável,0.00,0.0,2.0,9,265.0,1.0
684,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,Edílson,38523,290,1,-1.00,-1.0,lat,5.90,Provável,-2.10,3.0,3.0,9,294.0,0.0
685,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Chrigor,101398,280,1,2.40,0.0,ata,2.25,Nulo,0.00,1.0,1.0,9,276.0,1.0
686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Luiz Henrique,106987,373,0,0.00,0.0,lat,1.00,Nulo,0.00,1.0,2.0,9,267.0,1.0
687,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Abel Hernández,74159,285,2,0.55,1.2,ata,6.86,Provável,-0.03,0.0,2.0,9,354.0,0.0


In [307]:
df_jogadores.isna().sum()

A                       6898
CA                      5753
CV                      7475
DD                      7377
DP                      7621
DS                      3899
FC                      3888
FD                      5723
FF                      5150
FS                      4103
FT                      7274
G                       6729
GC                      7603
GS                      7365
I                       6621
PI                      3167
PP                      7594
SG                      6535
atletas.apelido            0
atletas.atleta_id          0
atletas.clube_id           0
atletas.jogos_num          0
atletas.media_num          0
atletas.pontos_num         0
atletas.posicao_id         0
atletas.preco_num          0
atletas.status_id          0
atletas.variacao_num       0
golContra                651
golFavor                 651
rodada_id                  0
adversario               651
jogoForaOuCasa           651
dtype: int64

In [314]:
# remover linhas cuja coluna rodada_id esteja NaN
df_jogadores.dropna(subset=['adversario'], inplace=True)

In [315]:
df_jogadores.isna().sum()

A                       3753
CA                      2683
CV                      4291
DD                      4204
DP                      4431
DS                       914
FC                       911
FD                      2649
FF                      2107
FS                      1106
FT                      4115
G                       3599
GC                      4416
GS                      4191
I                       3485
PI                       223
PP                      4407
SG                      3419
atletas.apelido            0
atletas.atleta_id          0
atletas.clube_id           0
atletas.jogos_num          0
atletas.media_num          0
atletas.pontos_num         0
atletas.posicao_id         0
atletas.preco_num          0
atletas.status_id          0
atletas.variacao_num       0
golContra                  0
golFavor                   0
rodada_id                  0
adversario                 0
jogoForaOuCasa             0
dtype: int64

In [317]:
df_jogadores.head(10)

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Léo Cittadini,70986,293,1,9.8,9.8,mei,12.90,Provável,4.90,0.0,2.0,1,356.0,1.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Matheus Bueno,101832,294,1,0.7,0.7,mei,1.23,Nulo,0.23,1.0,0.0,1,285.0,0.0
6,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Sassá,79578,294,1,-2.6,-2.6,ata,2.46,Nulo,-3.54,1.0,0.0,1,285.0,0.0
8,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,Matheus Galdezani,82634,294,1,5.9,5.9,mei,7.90,Provável,2.90,1.0,0.0,1,285.0,0.0
11,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,9.0,NaN,NaN,Diego Alves,38509,262,1,5.6,5.6,gol,14.21,Provável,0.21,1.0,0.0,1,282.0,0.0
12,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,Willian Arão,78478,262,1,5.2,5.2,mei,11.02,Provável,1.02,1.0,0.0,1,282.0,0.0
13,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,Rodrigo Caio,73800,262,1,0.7,0.7,zag,9.20,Provável,-2.80,1.0,0.0,1,282.0,0.0
14,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,Everton Ribeiro,51772,262,1,4.5,4.5,mei,6.89,Provável,1.89,1.0,0.0,1,282.0,0.0
15,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,12.0,NaN,NaN,Filipe Luís,38750,262,1,-4.7,-4.7,lat,3.84,Provável,-6.16,1.0,0.0,1,282.0,0.0
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Miguel,105612,266,1,0.3,0.3,mei,2.39,Nulo,-0.61,1.0,0.0,1,284.0,1.0


In [324]:
df_jogadores[(df_jogadores['atletas.clube_id'] == 262) & (df_jogadores['atletas.atleta_id'] == 38509)]

,A,CA,CV,DD,DP,DS,FC,FD,FF,FS,FT,G,GC,GS,I,PI,PP,SG,atletas.apelido,atletas.atleta_id,atletas.clube_id,atletas.jogos_num,atletas.media_num,atletas.pontos_num,atletas.posicao_id,atletas.preco_num,atletas.status_id,atletas.variacao_num,golContra,golFavor,rodada_id,adversario,jogoForaOuCasa
11,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,Diego Alves,38509,262,1,5.60,5.6,gol,14.21,Provável,0.21,1.0,0.0,1,282.0,0.0
53,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,19.0,0.0,1.0,Diego Alves,38509,262,5,2.22,0.0,gol,10.72,Contundido,0.00,2.0,0.0,10,354.0,1.0
110,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,9.0,0.0,0.0,Diego Alves,38509,262,2,-0.45,-6.5,gol,9.28,Suspenso,-4.93,3.0,0.0,2,373.0,1.0
386,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,9.0,0.0,0.0,Diego Alves,38509,262,2,-0.45,0.0,gol,9.28,Provável,0.00,0.0,1.0,3,294.0,1.0
128,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,13.0,0.0,0.0,Diego Alves,38509,262,3,-1.10,-2.4,gol,7.33,Provável,-1.95,1.0,1.0,4,284.0,0.0
28,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,14.0,0.0,0.0,Diego Alves,38509,262,4,-1.35,-2.1,gol,6.58,Provável,-0.75,1.0,1.0,5,263.0,0.0
681,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,19.0,0.0,1.0,Diego Alves,38509,262,5,2.22,16.5,gol,10.72,Dúvida,4.14,0.0,1.0,6,277.0,1.0
62,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,19.0,0.0,1.0,Diego Alves,38509,262,5,2.22,0.0,gol,10.72,Contundido,0.00,3.0,5.0,7,265.0,1.0
85,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,19.0,0.0,1.0,Diego Alves,38509,262,5,2.22,0.0,gol,10.72,Contundido,0.00,1.0,2.0,8,356.0,0.0
208,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,19.0,0.0,1.0,Diego Alves,38509,262,5,2.22,0.0,gol,10.72,Contundido,0.00,1.0,2.0,9,266.0,1.0


In [321]:
#df_jogadores[(df_jogadores['atletas.clube_id'] == 293) & (df_jogadores['atletas.atleta_id'] == 77544)]

<hr>

### Análise Descritiva
A análise de dados descritiva apresenta a separação e os tipos das características bem como quais colunas possuem valores vazios. Além de apresentar uma visão resumida da distribuição do dataset.

#### Features categóricas x numéricas
- categóricas:
    - nominal: ``TARGET``

- numéricas: 
    - continua: ``LAT``, ``LONG``
    - discreta: ``Movimentação``, ``Original_473``, ``Original_269``, ``Zero``, ``Maçã-Verde``, ``Tangerina``, ``Citrus``, ``Açaí-Guaraná``, ``Pêssego``,``row``
 



#### Tipos das características

- Float: 2 
- Int64: 10
- Objetos: 3 (Strings)


#### Dados faltantas / nulos
- Não existem dados faltantes no dataset

In [5]:
#Informações gerais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17016 entries, 0 to 17015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Tempo         17016 non-null  object 
 1   Estação       17016 non-null  object 
 2   LAT           17016 non-null  float64
 3   LONG          17016 non-null  float64
 4   Movimentação  17016 non-null  int64  
 5   Original_473  17016 non-null  int64  
 6   Original_269  17016 non-null  int64  
 7   Zero          17016 non-null  int64  
 8   Maçã-Verde    17016 non-null  int64  
 9   Tangerina     17016 non-null  int64  
 10  Citrus        17016 non-null  int64  
 11  Açaí-Guaraná  17016 non-null  int64  
 12  Pêssego       17016 non-null  int64  
 13  TARGET        17016 non-null  object 
dtypes: float64(2), int64(9), object(3)
memory usage: 1.8+ MB


<hr>

#### Distribuição numérica
- ``Original_473`` tem apróximadamente uma média 46 un.
- ``Original_473``,``Original_269``,``Zero`` já tiveram o mínimo de 0 un.

In [6]:
#Describe das variáveis numéricias
df.describe()

,LAT,LONG,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego
count,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000,17016.000000
mean,-23.572574,-46.652468,67674.478843,46.827045,36.036495,35.640691,24.339739,23.885696,24.428185,23.863540,23.904502
std,0.026361,0.033325,43791.284669,25.539867,18.787358,19.124799,12.089773,12.345912,12.072112,12.347261,12.308680
min,-23.626300,-46.725200,13592.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,-23.589500,-46.682300,33312.500000,25.000000,20.000000,19.000000,14.000000,13.000000,14.000000,13.000000,13.000000
50%,-23.566400,-46.648200,64194.500000,47.000000,36.000000,36.000000,24.000000,24.000000,25.000000,24.000000,24.000000
75%,-23.550000,-46.634600,89933.500000,68.000000,52.000000,52.000000,35.000000,35.000000,35.000000,35.000000,35.000000
max,-23.536500,-46.576500,195294.000000,86.000000,65.000000,65.000000,43.000000,43.000000,43.000000,43.000000,43.000000


#### Distribuição categórica
- A ``Estação`` Ana Rosa é a que mais possui registros
- No dia 24/08/2018 foi registrado a maior quantidade de registros (``tempo``)
- O maior quantidade ``target`` é o Normal 

In [7]:
#Describe das variáveis categóricas
df.describe(include=['O'])

,Tempo,Estação,TARGET
count,17016,17016,17016
unique,720,25,2
top,2018-8-24,Ana Rosa,NORMAL
freq,24,720,11518


<hr>


### Análise por características cruzadas

#### Features categóricas


In [8]:
#Total de registros por target
df['TARGET'].value_counts()

NORMAL         11518
REABASTECER     5498
Name: TARGET, dtype: int64

<hr>

### Análise por visualização de dados
Visualização para melhor interpretação e identificação de possíveis outliers.

<hr>

## Correção e Limpeza dos dados

### Desições tomadas após análise

- Remover as colunas ``row``, ``movimentacao``
- Adicionar mais amostras ao ``TARGET`` REABASTECER
- Visualizar matriz de correlação após correções.

### Pre-processando o dataset antes do treinamento

In [192]:
df = pd.read_csv('dataset.csv')
df_teste = pd.read_csv('to_be_scored.csv')

In [193]:
#total geral de unidades por linha
df['total_un'] = df.apply(lambda x: (x['Original_473'] + x['Original_269'] + x['Zero'] + x['Maçã-Verde'] + x['Tangerina'] + x['Citrus'] + x['Açaí-Guaraná'] + x['Pêssego'] ), axis=1)
df.loc[df['Tempo'] == '2018-2-30', 'Tempo'] = '2018-2-28'
df.loc[df['Tempo'] == '2018-2-29', 'Tempo'] = '2018-2-28'
df.loc[df['Tempo'] == '2019-2-30', 'Tempo'] = '2019-2-28'
df.loc[df['Tempo'] == '2019-2-29', 'Tempo'] = '2019-2-28'
df['Tempo'] = pd.to_datetime(df['Tempo'])


In [194]:
#Ajuste das labels
df = df.sort_values(['Estação','Tempo']).reset_index(drop=True)
for row in range(1, len(df)):
    if (row < 17014) and (df.loc[row, 'TARGET'] == 'NORMAL') and (df.loc[row+1, 'total_un'] > df.loc[row, 'total_un']):
        df.loc[row, 'TARGET'] = 'REABASTECER'


In [195]:
df = df.drop(['Tempo', 'Estação', 'Movimentação'],  axis=1)
df

,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET,total_un
0,-23.5813,-46.6383,86,65,65,43,43,43,43,43,NORMAL,431
1,-23.5813,-46.6383,75,60,55,41,37,43,37,37,NORMAL,385
2,-23.5813,-46.6383,68,54,45,38,31,40,31,31,NORMAL,338
3,-23.5813,-46.6383,57,45,37,37,25,35,25,29,NORMAL,290
4,-23.5813,-46.6383,45,42,30,36,23,32,20,24,NORMAL,252
...,...,...,...,...,...,...,...,...,...,...,...,...
17011,-23.5895,-46.6346,34,25,65,7,38,9,1,17,REABASTECER,196
17012,-23.5895,-46.6346,26,15,61,6,34,5,43,11,REABASTECER,201
17013,-23.5895,-46.6346,13,8,53,43,30,43,41,8,REABASTECER,239
17014,-23.5895,-46.6346,5,65,48,42,24,39,36,3,NORMAL,262


### Ajuste do DataFrame de Teste

In [197]:
#total geral de unidades por linha
df_teste['total_un'] = df_teste.apply(lambda x: (x['Original_473'] + x['Original_269'] + x['Zero'] + x['Maçã-Verde'] + x['Tangerina'] + x['Citrus'] + x['Açaí-Guaraná'] + x['Pêssego'] ), axis=1)
df_teste = df_teste.drop(['Tempo', 'Estação', 'Movimentação'],  axis=1)
df_teste.head(50)


,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,total_un
0,-23.5489,-46.6388,22,65,22,14,3,26,43,18,213
1,-23.5489,-46.6388,11,56,16,14,43,20,37,15,212
2,-23.5489,-46.6388,86,51,8,11,41,20,35,12,264
3,-23.5489,-46.6388,76,47,65,11,37,14,32,10,292
4,-23.5489,-46.6388,68,42,60,6,35,13,26,5,255
5,-23.5489,-46.6388,57,35,55,43,31,9,22,43,295
6,-23.5489,-46.6388,45,33,50,39,25,5,16,39,252
7,-23.5489,-46.6388,34,28,41,39,22,43,13,34,254
8,-23.5489,-46.6388,22,24,31,35,18,40,8,29,207
9,-23.5489,-46.6388,9,16,26,33,13,38,4,24,163


In [198]:
X_train = df.drop("TARGET", axis=1)
Y_train = df["TARGET"]
X_test = df_teste

In [200]:
#Balanceamento das classes com menos registros no dataset treino
smt = SMOTE()
X_train, Y_train = smt.fit_sample(X_train,Y_train)

In [201]:
#Normalização dos dados
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


### Treinando um classificador com base em no modelo Random Forest

In [212]:
#Algorítomo Machine Learning 
random_forest = RandomForestClassifier(n_estimators=80, max_depth=11, max_features=5, bootstrap=True, min_samples_leaf=3, min_samples_split=7)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

#importantes = pd.DataFrame(random_forest.feature_importances_, index= X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
#importantes
# Acurácia alcançada pela árvore de decisão
#print("Acurácia: {}%".format(100*round(accuracy_score(y_test, Y_pred), 2)))

In [213]:
# Acurácia alcançada pela árvore de decisão
round(random_forest.score(X_train, Y_train) * 100, 2)

96.04

### Submissão

In [210]:
result = df_teste.assign(TARGET=Y_pred)
result.head(50)

,LAT,LONG,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,total_un,TARGET
0,-23.5489,-46.6388,22,65,22,14,3,26,43,18,213,REABASTECER
1,-23.5489,-46.6388,11,56,16,14,43,20,37,15,212,REABASTECER
2,-23.5489,-46.6388,86,51,8,11,41,20,35,12,264,REABASTECER
3,-23.5489,-46.6388,76,47,65,11,37,14,32,10,292,NORMAL
4,-23.5489,-46.6388,68,42,60,6,35,13,26,5,255,REABASTECER
5,-23.5489,-46.6388,57,35,55,43,31,9,22,43,295,NORMAL
6,-23.5489,-46.6388,45,33,50,39,25,5,16,39,252,REABASTECER
7,-23.5489,-46.6388,34,28,41,39,22,43,13,34,254,NORMAL
8,-23.5489,-46.6388,22,24,31,35,18,40,8,29,207,NORMAL
9,-23.5489,-46.6388,9,16,26,33,13,38,4,24,163,REABASTECER


In [211]:
#Exportar submissão
result.to_csv('results.csv', index=False)